In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [3]:
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-3.3B")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-3.3B")

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/8.55G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/2.10G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
with open("floresp-v2.0-alpha.2/dev/dev.eng_Latn", encoding="UTF-8") as f:
    sentences = f.readlines()

translations = []

for sentence in sentences:
    inputs = tokenizer(sentence, return_tensors="pt")
    translated_tokens = model.generate(
        **inputs,
        forced_bos_token_id=tokenizer.lang_code_to_id["fra_Latn"],
        num_beams=1,
    )
    translation = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
    translations.append(translation)

translations = "\n".join(translations)

with open("hyp.fra_Latn", "w", encoding="UTF-8") as f:
    f.writelines(translations)

In [31]:
! python3.11 chrF++.py -R floresp-v2.0-alpha.2/dev/dev.ita_Latn -H translations/1_beam/hyp.ita_Latn

start_time:	1708305354
c6+w2-F2	56.1733
c6+w2-avgF2	56.1312
c6+w2-Prec	59.8474
c6+w2-Rec	55.3247
end_time:	1708305355


In [13]:
round(58.8396, 1)

58.8

In [4]:
! sacrebleu ../floresp-v2.0-rc.2/dev/dev.ukr_Cyrl -i translations/num_beams_1/hyp.ukr_Cyrl_1 -m bleu chrf ter --chrf-word-order 2

[
{
 "name": "BLEU",
 "score": 27.7,
 "signature": "nrefs:1|case:mixed|eff:no|tok:13a|smooth:exp|version:2.4.2",
 "verbose_score": "58.9/35.6/23.1/15.5 (BP = 0.941 ratio = 0.943 hyp_len = 20586 ref_len = 21841)",
 "nrefs": "1",
 "case": "mixed",
 "eff": "no",
 "tok": "13a",
 "smooth": "exp",
 "version": "2.4.2"
},
{
 "name": "chrF2++",
 "score": 52.6,
 "signature": "nrefs:1|case:mixed|eff:yes|nc:6|nw:2|space:no|version:2.4.2",
 "nrefs": "1",
 "case": "mixed",
 "eff": "yes",
 "nc": "6",
 "nw": "2",
 "space": "no",
 "version": "2.4.2"
},
{
 "name": "TER",
 "score": 59.7,
 "signature": "nrefs:1|case:lc|tok:tercom|norm:no|punct:yes|asian:no|version:2.4.2",
 "nrefs": "1",
 "case": "lc",
 "tok": "tercom",
 "norm": "no",
 "punct": "yes",
 "asian": "no",
 "version": "2.4.2"
}
]


In [ ]:
! comet-score -s floresp-v2.0-alpha.2/dev/dev.eng_Latn -t translations/1_beam/hyp.arb_Arab -r floresp-v2.0-alpha.2/dev/dev.arb_Arab